In [16]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn


In [96]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import csv
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB,GaussianNB
import os
from os.path import join
import datetime as dt


In [18]:
from scipy.stats.stats import pearsonr
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.cross_validation import StratifiedKFold
import itertools

In [19]:
df_message = pd.read_csv("./smsspamcollection/SMSSpamCollection",sep='\t', quoting=csv.QUOTE_NONE,
                           names=["label", "message"])

In [20]:
print df_message.shape
print df_message[:5]
print df_message.message[:5]


(5574, 2)
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: message, dtype: object


In [21]:
#test blob using ---------------------
message_test= "U dun say so early hor... U c already then say..."
message_test = unicode(message_test,'utf8').lower()
words = TextBlob(message_test).words
print words


[u'u', u'dun', u'say', u'so', u'early', u'hor', u'u', u'c', u'already', u'then', u'say']


In [22]:
print [word.lemma for word in words]
#end of test blob----------------------

[u'u', u'dun', u'say', u'so', u'early', u'hor', u'u', u'c', u'already', u'then', u'say']


In [23]:
def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [24]:
BOW_transform = CountVectorizer(analyzer=split_into_lemmas)
print BOW_transform

CountVectorizer(analyzer=<function split_into_lemmas at 0x10ba94050>,
        binary=False, charset=None, charset_error=None,
        decode_error=strict, dtype=<type 'numpy.int64'>, encoding=utf-8,
        input=content, lowercase=True, max_df=1.0, max_features=None,
        min_df=1, ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=(?u)\b\w\w+\b, tokenizer=None,
        vocabulary=None)


In [25]:
BOW_transform.fit(df_message['message'])

CountVectorizer(analyzer=<function split_into_lemmas at 0x10ba94050>,
        binary=False, charset=None, charset_error=None,
        decode_error=u'strict', dtype=<type 'numpy.int64'>,
        encoding=u'utf-8', input=u'content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None,
        vocabulary=None)

In [26]:
#using count vectorizer-------------
vectorizer = CountVectorizer(min_df=1)
print vectorizer

CountVectorizer(analyzer=word, binary=False, charset=None, charset_error=None,
        decode_error=strict, dtype=<type 'numpy.int64'>, encoding=utf-8,
        input=content, lowercase=True, max_df=1.0, max_features=None,
        min_df=1, ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=(?u)\b\w\w+\b, tokenizer=None,
        vocabulary=None)


In [27]:
content = ["How to format my hard disk", " Hard disk format problems "]
X = vectorizer.fit_transform(content)
print vectorizer.get_feature_names()
print(X.toarray().transpose())
#count vectorizer end--------------

[u'disk', u'format', u'hard', u'how', u'my', u'problems', u'to']
[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


In [31]:
print df_message['message'][2]
print df_message['label'][2]

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
spam


In [41]:
print BOW_transform.get_feature_names()[:5]


[u"''ok", u"'an", u"'anything", u"'comfort", u"'d"]


In [46]:
print BOW_transform.transform([df_message['message'][2]])
print 'length of the vector is:', BOW_transform.transform([df_message['message'][2]]).shape[1]
#The third message is transformed in count vectorizer and the postion of each word in the vectotrizer feauture names
# is shown and the count of that word in that sentence is displayed

  (0, 21)	2
  (0, 103)	1
  (0, 452)	1
  (0, 472)	1
  (0, 484)	1
  (0, 893)	1
  (0, 948)	1
  (0, 1272)	1
  (0, 1899)	1
  (0, 2266)	1
  (0, 2479)	1
  (0, 3046)	2
  (0, 3177)	2
  (0, 3287)	1
  (0, 3437)	1
  (0, 4183)	1
  (0, 5029)	1
  (0, 6297)	1
  (0, 6350)	1
  (0, 6408)	1
  (0, 7310)	1
  (0, 7533)	1
  (0, 7647)	1
  (0, 7801)	1
  (0, 7812)	3
  (0, 7996)	1
  (0, 8485)	1
  (0, 8525)	1
length of the vector is: 8874


In [48]:
# the postion of word "to" in the vector
print BOW_transform.get_feature_names()[7812]

#the word "to" appears 3 times in the message
print df_message['message'][2]

to
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [49]:
BOW = BOW_transform.transform(df_message['message'])

In [55]:
print 'Shape of the large transformed matrix is:', BOW.shape, 'i.e'
print BOW.shape[0], 'rows that corresponds to the number of messages and'
print BOW.shape[1], 'columns for each of the message that corresponds to the total number of unique words in the dataset'

Shape of the large transformed matrix is: (5574, 8874) i.e
5574 rows that corresponds to the number of messages and
8874 columns for each of the message that corresponds to the total number of unique words in the dataset


In [56]:
#normalizing and weighing
tfidf_BOW = TfidfTransformer().fit(BOW)

In [57]:
print tfidf_BOW.transform(BOW[2])


  (0, 8525)	0.179632904113
  (0, 8485)	0.138190739033
  (0, 7996)	0.118168109475
  (0, 7812)	0.171012810274
  (0, 7801)	0.208159499981
  (0, 7647)	0.11289854213
  (0, 7533)	0.133691560916
  (0, 7310)	0.185427055493
  (0, 6408)	0.157631945365
  (0, 6350)	0.157631945365
  (0, 6297)	0.158384636518
  (0, 5029)	0.148370514297
  (0, 4183)	0.0764098797291
  (0, 3437)	0.109085811318
  (0, 3287)	0.173494826823
  (0, 3177)	0.442847223391
  (0, 3046)	0.339376234237
  (0, 2479)	0.19016122188
  (0, 2266)	0.185427055493
  (0, 1899)	0.119671386057
  (0, 1272)	0.159958817058
  (0, 948)	0.0643973850704
  (0, 893)	0.208159499981
  (0, 484)	0.213953651361
  (0, 472)	0.213953651361
  (0, 452)	0.0944406763398
  (0, 103)	0.221423611696
  (0, 21)	0.183683813193


In [59]:
print 'inverse document frequency of the word "to" is:', tfidf_BOW.idf_[BOW_transform.vocabulary_['to']]


inverse document frequency of the word "to" is: 2.19534051378


In [60]:
#do it with entire bag of words
converted_messages = tfidf_BOW.transform(BOW)


In [75]:
print converted_messages.shape
#converted messages store the normalized weights of each word in a row of 8874 words
#and 5574 messages

(5574, 8874)


In [74]:
print converted_messages[1]
print converted_messages[1,8013]

  (0, 8471)	0.423369271093
  (0, 8013)	0.190655730508
  (0, 5660)	0.536156905844
  (0, 5628)	0.269618808649
  (0, 4641)	0.400512820409
  (0, 4431)	0.513653388721
0.190655730508


In [88]:
print converted_messages.dtype
y_train = np.asarray(df_message['label'], dtype="|S6")
print y_train.dtype
print df_message['label'].dtype

float64
|S6
object


In [89]:
split = int(0.75 * converted_messages.shape[0])
t1 = dt.datetime.now()
spam_detector = MultinomialNB().fit(converted_messages[:split], y_train[:split])
t2 = dt.datetime.now()
print 'total time taken in training:', (t2-t1)


total time taken in training: 0:00:00.007954


In [97]:
start = dt.datetime.now()
score = spam_detector.score(converted_messages[split:], y_train[split:])
end = dt.datetime.now()
print end-start
print score*100

0:00:00.005601
94.7632711621


In [99]:
from sklearn.ensemble import RandomForestClassifier

In [101]:
split = int(0.75 * converted_messages.shape[0])
t1 = dt.datetime.now()
spam_detector=RandomForestClassifier(n_estimators=200).fit(converted_messages[:split].toarray(), y_train[:split])
t2 = dt.datetime.now()
print 'total time taken in training:', (t2-t1)


total time taken in training: 0:02:51.821790


In [103]:
split = int(0.75 * converted_messages.shape[0])
t1 = dt.datetime.now()
spam_detector=RandomForestClassifier(n_estimators=400).fit(converted_messages[:split].toarray(), y_train[:split])
t2 = dt.datetime.now()
print 'total time taken in training:', (t2-t1)

total time taken in training: 0:06:08.771329


In [ ]:
split = int(0.75 * converted_messages.shape[0])
t1 = dt.datetime.now()
spam_detector=RandomForestClassifier(n_estimators=600).fit(converted_messages[:split].toarray(), y_train[:split])
t2 = dt.datetime.now()
print 'total time taken in training:', (t2-t1)

In [102]:
start = dt.datetime.now()
score = spam_detector.score(converted_messages[split:].toarray(), y_train[split:])
end = dt.datetime.now()
print end-start
print score*100

0:00:00.667909
97.0588235294
